# Search Engine Comparison Test

This notebook tests and compares various search engines available through LangChain.

**Tested Search Engines:**
1. **DuckDuckGo** - Privacy-focused, **no API key required** ✓
2. **SearxNG** - Meta search engine, **no API key required** ✓
3. **Search1API** - Flexible search API, **optional API key** (can try without)
4. **Mojeek** - Independent privacy search (requires API key)
5. **Tavily** - AI-optimized search (requires API key, ⚠️ quality concerns)

**Focus:** This notebook emphasizes search engines that work without API keys or with minimal setup.

**Setup Requirements:**
```bash
pip install langchain-community duckduckgo-search ddgs tavily-python langchain-search1api
```

In [ ]:
# Import required libraries
import os
import time
from typing import List, Dict, Any
from datetime import datetime
import json

# LangChain search utilities
from langchain_community.utilities import (
    DuckDuckGoSearchAPIWrapper,
    TavilySearchAPIWrapper,
    SearxSearchWrapper
)

# Mojeek search
try:
    from langchain_community.tools.mojeek_search import MojeekSearch
    MOJEEK_AVAILABLE = True
except ImportError:
    print("⚠ Mojeek not available - install with: pip install langchain-community")
    MOJEEK_AVAILABLE = False

# Search1API
try:
    from langchain_search1api import SearchTool
    SEARCH1API_AVAILABLE = True
except ImportError:
    print("⚠ Search1API not available - install with: pip install langchain-search1api")
    SEARCH1API_AVAILABLE = False

print("✓ Libraries imported successfully")

## Configuration

Set up API keys for search engines that require them.

In [ ]:
# API Keys Configuration
# Only needed for: Tavily, Mojeek, and optionally Search1API

TAVILY_API_KEY = os.getenv('TAVILY_API_KEY', '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')
MOJEEK_API_KEY = os.getenv('MOJEEK_API_KEY', '')  # Get from https://www.mojeek.com/services/search/web-search-api/
SEARCH1API_KEY = os.getenv('SEARCH1API_KEY', '')  # Optional - can use without key
SEARX_HOST = os.getenv('SEARX_HOST', 'https://searx.be')  # Public SearxNG instance

# Test query
TEST_QUERY = "What are the latest developments in artificial intelligence 2025?"
MAX_RESULTS = 5

print(f"Test Query: {TEST_QUERY}")
print(f"Max Results: {MAX_RESULTS}")
print(f"\nSearch Engines Status:")
print(f"  ✓ DuckDuckGo: Ready (no API key needed)")
print(f"  ✓ SearxNG: Ready (using {SEARX_HOST})")
print(f"  {'✓' if SEARCH1API_AVAILABLE else '✗'} Search1API: {'Ready' if SEARCH1API_AVAILABLE else 'Not installed'} (API key optional)")
print(f"  {'✓' if MOJEEK_AVAILABLE else '✗'} Mojeek: {'Ready' if MOJEEK_AVAILABLE else 'Not installed'} - API key {'set' if MOJEEK_API_KEY else 'needed'}")
print(f"  {'✓' if TAVILY_API_KEY else '✗'} Tavily: {'Ready' if TAVILY_API_KEY else 'API key needed'} ⚠️ Quality concerns")

## Helper Functions

In [ ]:
def format_results(results: List[Dict], engine_name: str) -> None:
    """Pretty print search results"""
    print(f"\n{'='*80}")
    print(f"Results from {engine_name}")
    print(f"{'='*80}")
    
    if not results:
        print("No results found.")
        return
    
    for i, result in enumerate(results[:MAX_RESULTS], 1):
        print(f"\n{i}. {result.get('title', 'No title')}")
        print(f"   URL: {result.get('link', result.get('url', 'No URL'))}")
        snippet = result.get('snippet', result.get('content', result.get('description', 'No description')))
        print(f"   {snippet[:200]}..." if len(snippet) > 200 else f"   {snippet}")
    
    print(f"\n{'='*80}\n")

def time_search(func, *args, **kwargs):
    """Time a search function execution"""
    start = time.time()
    try:
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        return result, elapsed, None
    except Exception as e:
        elapsed = time.time() - start
        return None, elapsed, str(e)

print("✓ Helper functions defined")

## 1. DuckDuckGo Search (No API Key Required)

DuckDuckGo is privacy-focused and doesn't require an API key, making it great for testing.

In [ ]:
print("Testing DuckDuckGo Search...\n")

ddg_search = DuckDuckGoSearchAPIWrapper(max_results=MAX_RESULTS)
results, elapsed, error = time_search(ddg_search.results, TEST_QUERY, MAX_RESULTS)

if error:
    print(f"❌ DuckDuckGo Error: {error}")
else:
    print(f"✓ Completed in {elapsed:.2f}s")
    format_results(results, "DuckDuckGo")
    
    # Store for comparison
    ddg_results = results

## 2. Tavily Search (AI-Optimized) ⚠️ Quality Concerns

Tavily is optimized for AI applications but has known quality issues:
- **2x larger token usage** vs competitors (1928 vs 918 tokens)
- **Includes low-relevant info** which increases costs
- **Requires post-processing** to filter results
- Best used with additional LLM filtering

Despite issues, included since you already have an API key.

In [ ]:
if TAVILY_API_KEY:
    print("Testing Tavily Search...\\n")
    print("⚠️ Note: Tavily may return larger, less focused results")
    
    tavily_search = TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY)
    results, elapsed, error = time_search(tavily_search.results, TEST_QUERY, MAX_RESULTS)
    
    if error:
        print(f"❌ Tavily Error: {error}")
    else:
        print(f"✓ Completed in {elapsed:.2f}s")
        
        # Show token count warning
        if results:
            total_chars = sum(len(str(r.get('content', ''))) for r in results)
            print(f"⚠️ Total content size: {total_chars} chars (may be larger than needed)")
        
        format_results(results, "Tavily")
        
        # Store for comparison
        tavily_results = results
else:
    print("⚠ Tavily API key not set, skipping...")

## 3. SearxNG (Meta Search Engine - No API Key)

SearxNG aggregates results from multiple search engines for maximum coverage.

In [ ]:
print("Testing SearxNG Search...\\n")

try:
    searx_search = SearxSearchWrapper(searx_host=SEARX_HOST)
    results, elapsed, error = time_search(searx_search.results, TEST_QUERY, MAX_RESULTS)
    
    if error:
        print(f"❌ SearxNG Error: {error}")
    else:
        print(f"✓ Completed in {elapsed:.2f}s")
        format_results(results, "SearxNG")
        
        # Store for comparison
        searx_results = results
except Exception as e:
    print(f"⚠ SearxNG not available: {e}")

## 4. Search1API (Optional API Key)

Search1API allows testing without an API key, or use a free key for higher limits.

In [ ]:
if SEARCH1API_AVAILABLE:
    print("Testing Search1API...\\n")
    
    try:
        # Can initialize with or without API key
        if SEARCH1API_KEY:
            search1_tool = SearchTool(api_key=SEARCH1API_KEY)
            print("Using Search1API with API key")
        else:
            search1_tool = SearchTool()
            print("Using Search1API without API key (limited)")
        
        # Search1API returns results in a different format
        results, elapsed, error = time_search(lambda: search1_tool.run(TEST_QUERY))
        
        if error:
            print(f"❌ Search1API Error: {error}")
        else:
            print(f"✓ Completed in {elapsed:.2f}s")
            print(f"\\nSearch1API Results:\\n{results[:500]}...")
            
    except Exception as e:
        print(f"❌ Search1API Error: {e}")
else:
    print("⚠ Search1API not installed. Install with: pip install langchain-search1api")

## 5. Mojeek Search (Independent Privacy Search)

Mojeek is an independent search engine with its own index, focused on privacy.

In [ ]:
if MOJEEK_AVAILABLE and MOJEEK_API_KEY:
    print("Testing Mojeek Search...\\n")
    
    try:
        # Mojeek uses LangChain Tool interface
        mojeek_search = MojeekSearch.config(
            api_key=MOJEEK_API_KEY,
            search_kwargs={"t": MAX_RESULTS}  # t = number of results
        )
        
        results, elapsed, error = time_search(lambda: mojeek_search.run(TEST_QUERY))
        
        if error:
            print(f"❌ Mojeek Error: {error}")
        else:
            print(f"✓ Completed in {elapsed:.2f}s")
            print(f"\\nMojeek Results:\\n{results[:500]}...")
            
    except Exception as e:
        print(f"❌ Mojeek Error: {e}")
        print("Get API key from: https://www.mojeek.com/services/search/web-search-api/")
elif not MOJEEK_AVAILABLE:
    print("⚠ Mojeek not available. Already included in langchain-community")
else:
    print("⚠ Mojeek API key not set")
    print("Get API key from: https://www.mojeek.com/services/search/web-search-api/")

## Performance Comparison

Let's compare the performance and quality of the 5 search engines we tested.

In [ ]:
# Removed - Performance comparison moved to cell-19

## Performance Comparison Results

Comparing all 5 search engines on speed, results quality, and ease of use.

In [ ]:
# Collect metrics from all search engines
comparison_data = []

# Test each engine with timing - only the 5 we're using
engines_to_test = [
    ("DuckDuckGo", lambda: DuckDuckGoSearchAPIWrapper(max_results=MAX_RESULTS).results(TEST_QUERY, MAX_RESULTS)),
]

# Add SearxNG (no API key needed)
engines_to_test.append(
    ("SearxNG", lambda: SearxSearchWrapper(searx_host=SEARX_HOST).results(TEST_QUERY, MAX_RESULTS))
)

# Add Tavily if API key available
if TAVILY_API_KEY:
    engines_to_test.append(
        ("Tavily", lambda: TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY).results(TEST_QUERY, MAX_RESULTS))
    )

print("\\n" + "="*80)
print("PERFORMANCE COMPARISON - 5 Search Engines")
print("="*80)
print(f"\\n{'Engine':<20} {'Time (s)':<12} {'Results':<10} {'Status'}{'Notes':<30}")
print("-" * 100)

for engine_name, search_func in engines_to_test:
    results, elapsed, error = time_search(search_func)
    
    notes = ""
    if engine_name == "Tavily":
        notes = "⚠️ Quality concerns"
    elif engine_name == "DuckDuckGo":
        notes = "✓ No API key"
    elif engine_name == "SearxNG":
        notes = "✓ No API key, Meta-search"
    
    if error:
        print(f"{engine_name:<20} {elapsed:<12.2f} {'0':<10} ❌ Error {notes}")
        comparison_data.append({
            'engine': engine_name,
            'time': elapsed,
            'results': 0,
            'status': 'error',
            'error': error,
            'notes': notes
        })
    else:
        num_results = len(results) if results else 0
        print(f"{engine_name:<20} {elapsed:<12.2f} {num_results:<10} ✓ Success {notes}")
        comparison_data.append({
            'engine': engine_name,
            'time': elapsed,
            'results': num_results,
            'status': 'success',
            'notes': notes
        })

print("\\n" + "="*80)

## Summary and Recommendations

In [ ]:
print("\\n" + "="*80)
print("SEARCH ENGINE COMPARISON SUMMARY")
print("="*80)

print("\\n📊 Performance Metrics:")
successful = [d for d in comparison_data if d['status'] == 'success']
if successful:
    fastest = min(successful, key=lambda x: x['time'])
    print(f"  Fastest: {fastest['engine']} ({fastest['time']:.2f}s)")
    
    most_results = max(successful, key=lambda x: x['results'])
    print(f"  Most Results: {most_results['engine']} ({most_results['results']} results)")

print("\\n📝 Recommendations by Priority:")

print("\\n  🥇 1. DuckDuckGo (BEST FOR GETTING STARTED):")
print("     ✓ No API key required")
print("     ✓ Privacy-focused")
print("     ✓ Fast and reliable")
print("     ✓ Good for development/testing")
print("     ⚠ May have rate limits")

print("\\n  🥈 2. SearxNG (BEST FOR COVERAGE):")
print("     ✓ No API key required")
print("     ✓ Meta-search (aggregates multiple engines)")
print("     ✓ Maximum result diversity")
print("     ✓ Privacy-focused")
print("     ⚠ Depends on public instance availability")

print("\\n  🥉 3. Search1API (FLEXIBLE OPTION):")
print("     ✓ Can test without API key")
print("     ✓ Optional free key for higher limits")
print("     ✓ Good balance of features")
print("     ⚠ Requires installation of langchain-search1api")

print("\\n  4. Mojeek (INDEPENDENT INDEX):")
print("     ✓ Independent search engine (own index)")
print("     ✓ Privacy-focused")
print("     ✓ Not reliant on other search engines")
print("     ⚠ Requires API key")

print("\\n  ⚠️ 5. Tavily (USE WITH CAUTION):")
print("     ❌ 2x larger token usage vs competitors")
print("     ❌ Includes low-relevant information")
print("     ❌ Higher costs due to verbose results")
print("     ⚠ Requires post-processing/filtering")
print("     ✓ Optimized for AI (in theory)")

print("\\n🎯 Best Choice for Your Use Case:")
print("  • Just Starting / Development: DuckDuckGo (instant, no setup)")
print("  • Maximum Coverage: SearxNG (meta-search)")
print("  • Privacy + Independent: Mojeek (own index)")
print("  • Flexible Testing: Search1API (optional key)")
print("  • ❌ Avoid for cost-sensitive: Tavily (quality issues)")

print("\\n💡 Recommended Strategy:")
print("  1. Start with DuckDuckGo (always works, no key)")
print("  2. Add SearxNG for broader coverage")
print("  3. Implement fallback: DuckDuckGo → SearxNG → Others")
print("  4. Avoid Tavily unless you need AI-specific features AND")
print("     are willing to post-process/filter results")

print("\\n" + "="*80)

# Save comparison data
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"search_comparison_{timestamp}.json"
with open(output_file, 'w') as f:
    json.dump({
        'timestamp': timestamp,
        'test_query': TEST_QUERY,
        'max_results': MAX_RESULTS,
        'engines_tested': len(comparison_data),
        'results': comparison_data
    }, f, indent=2)

print(f"\\n💾 Results saved to: {output_file}")

## Advanced: Multi-Engine Fallback Strategy

Example of using multiple search engines with intelligent fallback (prioritizing free options).

In [ ]:
def multi_engine_search(query: str, max_results: int = 5) -> Dict[str, Any]:
    """
    Search using multiple engines with intelligent fallback
    Priority: Free options first, paid options as fallback
    """
    all_results = {}
    
    # Priority 1: DuckDuckGo (always available, no API key)
    try:
        ddg = DuckDuckGoSearchAPIWrapper(max_results=max_results)
        all_results['duckduckgo'] = ddg.results(query, max_results)
        print("✓ DuckDuckGo: Success")
    except Exception as e:
        all_results['duckduckgo'] = {'error': str(e)}
        print(f"✗ DuckDuckGo: {e}")
    
    # Priority 2: SearxNG (no API key, meta-search)
    try:
        searx = SearxSearchWrapper(searx_host=SEARX_HOST)
        all_results['searxng'] = searx.results(query, max_results)
        print("✓ SearxNG: Success")
    except Exception as e:
        all_results['searxng'] = {'error': str(e)}
        print(f"✗ SearxNG: {e}")
    
    # Priority 3: Tavily (only if API key available, despite quality issues)
    if TAVILY_API_KEY:
        try:
            tavily = TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY)
            results = tavily.results(query, max_results)
            all_results['tavily'] = results
            print("⚠️ Tavily: Success (but results may be verbose)")
        except Exception as e:
            all_results['tavily'] = {'error': str(e)}
            print(f"✗ Tavily: {e}")
    
    return all_results

# Test multi-engine search
print("Testing multi-engine fallback strategy...\\n")
multi_results = multi_engine_search(TEST_QUERY, MAX_RESULTS)

print(f"\\nEngines attempted: {list(multi_results.keys())}")
successful_engines = [k for k, v in multi_results.items() if not isinstance(v, dict) or 'error' not in v]
print(f"Successful engines: {successful_engines}")
print(f"Total results: {sum(len(v) if isinstance(v, list) else 0 for v in multi_results.values())}")

print("\\n💡 Recommendation: Use DuckDuckGo as primary, SearxNG as fallback")
print("   This gives you redundancy without needing API keys!")

## Conclusion

This notebook tested 5 search engines with emphasis on free, no-API-key options.

**Key Takeaways:**
- **DuckDuckGo is the winner** for getting started (no API key, reliable)
- **SearxNG provides excellent coverage** as a meta-search engine (no API key)
- **Tavily has quality issues** - 2x token usage, low-relevant info, avoid unless necessary
- **Search1API and Mojeek** are good alternatives with different trade-offs

**Recommended Setup:**
```python
# Production-ready fallback strategy
engines = [
    DuckDuckGoSearchAPIWrapper(max_results=5),  # Primary
    SearxSearchWrapper(searx_host="https://searx.be"),  # Fallback
]

for engine in engines:
    try:
        results = engine.results(query, max_results)
        if results:
            return results  # Use first successful result
    except:
        continue  # Try next engine
```

**Next Steps:**
1. ✓ Start with DuckDuckGo (already working, no setup)
2. ✓ Add SearxNG for redundancy (no setup needed)
3. Get API keys only if you need:
   - Mojeek (independent search index)
   - Search1API (flexible limits)
4. ❌ Avoid Tavily unless you specifically need AI features AND can handle verbose results

**Cost Savings:**
By using DuckDuckGo + SearxNG, you get robust search with **zero API costs** and **no API key management**!